In [1]:
import pandas as pd
# from utils import select_min_pay, read_input_file, export_file
import glob
import numpy as np
import pandas as pd
import glob
import logging
import win32com.client as win32
import os
import jaydebeapi
from const import QUERY, CONNECT
import oracledb
import getpass
import json

history weight

In [277]:
conn = jaydebeapi.connect(
        CONNECT.DB,
        CONNECT.PORT,
        CONNECT.USER,
        CONNECT.CNN,
        )
cur = conn.cursor()  

query = pd.read_sql(f'''
      SELECT * FROM SIREETRON.bucket_score  
    ''', conn) 
conn.close()

C:\Users\Sireetron\AppData\Local\Temp\ipykernel_23336\996401911.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  query = pd.read_sql(f'''


In [278]:
query

,ID,BUCKET,SCORE_INDEX,BUCKET_SCORE
0,1,99:NORMAL,1,0
1,2,1:XD,1,1
2,3,2:OD1,2,2
3,4,3:OD2,2,3
4,5,4:OD3,3,4
5,6,5:OD4,3,5
6,7,6:OD5,3,6
7,8,7:OD6,3,7
8,9,8:OD7,3,8
9,10,9:OD8,3,9


In [280]:

def all_history(date,item) :
    data_mnt = pd.read_csv(f'./input/acc_history_monthly/Acc_{date}_0830.csv')
    print(f'./input/history/Acc_{date}_0830.csv')
    data_mnt['Loan No'] = data_mnt['Loan No'].astype(str)
    data_mnt = data_mnt[['Loan No','OverdueCnt_Morning']].merge(query, left_on='OverdueCnt_Morning', right_on='BUCKET', how='left')
    data_mnt = data_mnt.rename(columns={'BUCKET_SCORE': f'BUCKET_SCORE{item}mnt'})
    # data_mnt = data_mnt.rename(columns={'OverdueCnt_Morning': f'OverdueCnt_Morning{item}'})
    return data_mnt[['Loan No',f'BUCKET_SCORE{item}mnt']]



In [281]:
data_digit1 = pd.read_csv(f'./input/acc_current/Acc_20250301_0830.csv')
data_digit1['Loan No'] = data_digit1['Loan No'].astype(str)
data_digit1 = data_digit1[['Loan No']]

In [282]:
his = [ ['20250301', '0'],['20250201', '1'],['20250101','2'],['20241201','3'],['20241101','4'],['20241001','5']]
for i in his[0:]:
    print(f"Processing date: {i[0]}")
    print((i[0], i[1]))
    data = all_history(i[0], i[1])
    data_digit1 = data_digit1.merge(data, on='Loan No', how='outer')

Processing date: 20250301
('20250301', '0')
./input/history/Acc_20250301_0830.csv
Processing date: 20250201
('20250201', '1')
./input/history/Acc_20250201_0830.csv
Processing date: 20250101
('20250101', '2')
./input/history/Acc_20250101_0830.csv
Processing date: 20241201
('20241201', '3')
./input/history/Acc_20241201_0830.csv
Processing date: 20241101
('20241101', '4')


C:\Users\Sireetron\AppData\Local\Temp\ipykernel_23336\2985231876.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data_mnt = pd.read_csv(f'./input/acc_history_monthly/Acc_{date}_0830.csv')


./input/history/Acc_20241101_0830.csv
Processing date: 20241001
('20241001', '5')


C:\Users\Sireetron\AppData\Local\Temp\ipykernel_23336\2985231876.py:2: DtypeWarning: Columns (1,6) have mixed types. Specify dtype option on import or set low_memory=False.
  data_mnt = pd.read_csv(f'./input/acc_history_monthly/Acc_{date}_0830.csv')


./input/history/Acc_20241001_0830.csv


In [286]:
data_digit1['AVG_DIGIT1'] = data_digit1.loc[:, data_digit1.columns.str.startswith('BUCKET_SCORE')].apply(pd.to_numeric, errors='coerce').sum(axis=1).fillna(0).astype(int)/6

In [288]:
data_digit1

,Loan No,BUCKET_SCORE0mnt,BUCKET_SCORE1mnt,BUCKET_SCORE2mnt,BUCKET_SCORE3mnt,BUCKET_SCORE4mnt,BUCKET_SCORE5mnt,AVG_DIGIT1
0,1000158080014811,0,0,0,0,0,0,0.0
1,1000158090005394,0,0,0,0,0,0,0.0
2,1000158090008580,0,0,0,0,0,0,0.0
3,1000158090014814,0,0,0,0,0,0,0.0
4,1000158090014831,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...
1633098,9010221122000914,13,13,13,13,13,13,13.0
1633099,9010221122000922,13,13,13,13,13,13,13.0
1633100,9010221122000931,0,0,0,0,0,0,0.0
1633101,9010221122000949,5,5,5,5,5,5,5.0


In [ ]:
# conditions = [
#     (data_digit1['AVG_DIGIT1_Fhn'] > 3),
#     (data_digit1['AVG_DIGIT1_Fhn'] > 1),
#     (data_digit1['AVG_DIGIT1_Fhn'] == 0)
# ]

# # Define corresponding values
# choices = ['H', 'M', 'L']

# # Apply the mapping logic
# data_digit1['AVG_DIGIT1_digit_Fhn'] = np.select(conditions, choices, default='M')

In [ ]:
# data_digit1[]

,Loan No,BUCKET_SCORE0mnt,BUCKET_SCORE1mnt,BUCKET_SCORE2mnt,BUCKET_SCORE3mnt,BUCKET_SCORE4mnt,BUCKET_SCORE5mnt
0,1000158080014811,0,0,0,0,0,0
1,1000158090005394,0,0,0,0,0,0
2,1000158090008580,0,0,0,0,0,0
3,1000158090014814,0,0,0,0,0,0
4,1000158090014831,0,0,0,0,0,0
...,...,...,...,...,...,...,...
1633098,9010221122000914,13,13,13,13,13,13
1633099,9010221122000922,13,13,13,13,13,13
1633100,9010221122000931,0,0,0,0,0,0
1633101,9010221122000949,5,5,5,5,5,5


In [182]:
data_digit1.loc[data_digit1['Loan No'] == '9010221122000949']

,Loan No,SCORE_INDEX0mnt_x,SCORE_INDEX1mnt_x,SCORE_INDEX2mnt_x,SCORE_INDEX3mnt_x,SCORE_INDEX4mnt_x,SCORE_INDEX5mnt_x,SCORE_INDEX0mnt_y,SCORE_INDEX1mnt_y,SCORE_INDEX2mnt_y,SCORE_INDEX3mnt_y,SCORE_INDEX4mnt_y,SCORE_INDEX5mnt_y
1633101,9010221122000949,3,3,3,3,3,3,3,3,3,3,3,3


In [252]:
import pandas as pd

def all_history(date, item):
    """Fetch and process historical account data for a given date."""
    
    file_path = f'./input/acc_history_monthly/Acc_{date}_0830.csv'
    print(f"Processing file: {file_path}")
    
    data_mnt = pd.read_csv(file_path)
    data_mnt['Loan No'] = data_mnt['Loan No'].astype(str)
    
    # Merge with query data on 'OverdueCnt_Morning' and rename columns
    data_mnt = data_mnt[['Loan No', 'OverdueCnt_Morning']].merge(
        query, left_on='OverdueCnt_Morning', right_on='BUCKET', how='left'
    )
    
    data_mnt = data_mnt.rename(columns={
        'SCORE_INDEX': f'SCORE_INDEX{item}mnt',
        'OverdueCnt_Morning': f'OverdueCnt_Morning{item}',
        'BUCKET_SCORE': f'BUCKET_SCORE{item}mnt'
    })
    
    return data_mnt[['Loan No', f'OverdueCnt_Morning{item}', f'SCORE_INDEX{item}mnt',f'BUCKET_SCORE{item}mnt']]

# Load current account data
data_digit1 = pd.read_csv('./input/acc_current/Acc_20250301_0830.csv')
data_digit1['Loan No'] = data_digit1['Loan No'].astype(str)
data_digit1 = data_digit1[['Loan No']]

# Define historical dates and corresponding labels
history_dates = [
    ('20250301', '0'), ('20250201', '1'), ('20250101', '2'),
    ('20241201', '3'), ('20241101', '4'), ('20241001', '5')
]

# Process historical data and merge with current data
for date, item in history_dates:
    print(f"Processing date: {date}")
    data = all_history(date, item)
    data_digit1 = data_digit1.merge(data, on='Loan No', how='outer')

# Final dataset is stored in `data_digit1`


Processing date: 20250301
Processing file: ./input/acc_history_monthly/Acc_20250301_0830.csv
Processing date: 20250201
Processing file: ./input/acc_history_monthly/Acc_20250201_0830.csv
Processing date: 20250101
Processing file: ./input/acc_history_monthly/Acc_20250101_0830.csv
Processing date: 20241201
Processing file: ./input/acc_history_monthly/Acc_20241201_0830.csv
Processing date: 20241101
Processing file: ./input/acc_history_monthly/Acc_20241101_0830.csv


C:\Users\Sireetron\AppData\Local\Temp\ipykernel_23336\1505450073.py:9: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data_mnt = pd.read_csv(file_path)


Processing date: 20241001
Processing file: ./input/acc_history_monthly/Acc_20241001_0830.csv


C:\Users\Sireetron\AppData\Local\Temp\ipykernel_23336\1505450073.py:9: DtypeWarning: Columns (1,6) have mixed types. Specify dtype option on import or set low_memory=False.
  data_mnt = pd.read_csv(file_path)


In [253]:
data_digit1

,Loan No,OverdueCnt_Morning0,SCORE_INDEX0mnt,BUCKET_SCORE0mnt,OverdueCnt_Morning1,SCORE_INDEX1mnt,BUCKET_SCORE1mnt,OverdueCnt_Morning2,SCORE_INDEX2mnt,BUCKET_SCORE2mnt,OverdueCnt_Morning3,SCORE_INDEX3mnt,BUCKET_SCORE3mnt,OverdueCnt_Morning4,SCORE_INDEX4mnt,BUCKET_SCORE4mnt,OverdueCnt_Morning5,SCORE_INDEX5mnt,BUCKET_SCORE5mnt
0,1000158080014811,99:NORMAL,1,0,99:NORMAL,1,0,99:NORMAL,1,0,99:NORMAL,1,0,99:NORMAL,1,0,99:NORMAL,1,0
1,1000158090005394,99:NORMAL,1,0,99:NORMAL,1,0,99:NORMAL,1,0,99:NORMAL,1,0,99:NORMAL,1,0,99:NORMAL,1,0
2,1000158090008580,99:NORMAL,1,0,99:NORMAL,1,0,99:NORMAL,1,0,99:NORMAL,1,0,99:NORMAL,1,0,99:NORMAL,1,0
3,1000158090014814,99:NORMAL,1,0,99:NORMAL,1,0,99:NORMAL,1,0,99:NORMAL,1,0,99:NORMAL,1,0,99:NORMAL,1,0
4,1000158090014831,99:NORMAL,1,0,99:NORMAL,1,0,99:NORMAL,1,0,99:NORMAL,1,0,99:NORMAL,1,0,99:NORMAL,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1633098,9010221122000914,98:Write-Off,3,13,98:Write-Off,3,13,98:Write-Off,3,13,98:Write-Off,3,13,98:Write-Off,3,13,98:Write-Off,3,13
1633099,9010221122000922,98:Write-Off,3,13,98:Write-Off,3,13,98:Write-Off,3,13,98:Write-Off,3,13,98:Write-Off,3,13,98:Write-Off,3,13
1633100,9010221122000931,99:NORMAL,1,0,99:NORMAL,1,0,99:NORMAL,1,0,99:NORMAL,1,0,99:NORMAL,1,0,99:NORMAL,1,0
1633101,9010221122000949,5:OD4,3,5,5:OD4,3,5,5:OD4,3,5,5:OD4,3,5,5:OD4,3,5,5:OD4,3,5


In [270]:
# Define conditions
conditions = [
    (data_digit1['AVG_DIGIT1_PK'] > 3),
    (data_digit1['AVG_DIGIT1_PK'] > 1),
    (data_digit1['AVG_DIGIT1_PK'] >= 0)
]

# Define corresponding values
choices = ['H', 'M', 'L']

# Apply the mapping logic
data_digit1['AVG_DIGIT1_digit_Pk'] = np.select(conditions, choices, default='aaaaaa')  # Default is 'M' for values in between

# Display updated DataFrame
# print(data_digit1[['AVG_DIGIT1_Fhn', 'AVG_DIGIT1_digit_Fhn']])

In [273]:
data_digit1 = data_digit1.loc[data_digit1['AVG_DIGIT1_digit_Fhn'] != data_digit1['AVG_DIGIT1_digit_Pk'] ]

In [289]:
data_digit1['Loan No'] = data_digit1['Loan No'].astype(str)

In [290]:
data_digit1

,Loan No,BUCKET_SCORE0mnt,BUCKET_SCORE1mnt,BUCKET_SCORE2mnt,BUCKET_SCORE3mnt,BUCKET_SCORE4mnt,BUCKET_SCORE5mnt,AVG_DIGIT1
0,1000158080014811,0,0,0,0,0,0,0.0
1,1000158090005394,0,0,0,0,0,0,0.0
2,1000158090008580,0,0,0,0,0,0,0.0
3,1000158090014814,0,0,0,0,0,0,0.0
4,1000158090014831,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...
1633098,9010221122000914,13,13,13,13,13,13,13.0
1633099,9010221122000922,13,13,13,13,13,13,13.0
1633100,9010221122000931,0,0,0,0,0,0,0.0
1633101,9010221122000949,5,5,5,5,5,5,5.0


Perfomance

In [291]:
data_performance = pd.read_csv('./input/performance/performance.csv')
# data_AC = pd.read_csv('./input/Acc_20250307_0830.csv')

C:\Users\Sireetron\AppData\Local\Temp\ipykernel_23336\2899921485.py:1: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  data_performance = pd.read_csv('./input/performance/performance.csv')


In [292]:
data_performance

,AS_OF_DATE,CONTRACT_NO,NATIONAL_ID,GROSS_INCOME,FLOORED_VALUE,OCCUPATION_TYPE,OCCUPATION_NAME,CONTRACT_DATE,CONTRACT_PERIOD,AGE,PRINCIPAL_BAL,MOB,DIGIT2INCOME,DIGIT2AGE,DIGIT2JOB,TOTAL_DIGIT2
0,2025-03-09 00:00:00.000,7230950101266462,1709700073957,30000.0,30000.0,5.0,OFFICE WORK,2023-09-21 00:00:00.000,17.0,36.0,47052.21,7,3,3,1,3
1,2025-03-09 00:00:00.000,5000161041003419,3300800674709,15611.0,15000.0,9.0,PRODUCTION LINE LABOR,2018-04-02 00:00:00.000,83.0,47.0,0.00,7,3,2,3,3
2,2025-03-09 00:00:00.000,7230410100429114,1401500153197,12224.0,10000.0,9.0,PRODUCTION LINE LABOR,2023-04-25 00:00:00.000,22.0,30.0,274168.44,7,3,3,3,3
3,2025-03-09 00:00:00.000,5000461011069133,1102001592652,10000.0,10000.0,6.0,SALES,2018-01-29 00:00:00.000,85.0,33.0,0.00,7,3,3,2,3
4,2025-03-09 00:00:00.000,5000159121088434,1100200920712,10250.0,10000.0,7.0,DRIVING,2016-12-28 00:00:00.000,98.0,30.0,0.00,7,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5323431,2025-03-09 00:00:00.000,7250270104833429,1910100097898,45000.0,45000.0,5.0,OFFICE WORK,2025-02-23 00:00:00.000,0.0,37.0,0.00,6,2,3,1,2
5323432,2025-03-09 00:00:00.000,7250270104828777,1600100045670,17500.0,15000.0,5.0,OFFICE WORK,2025-02-23 00:00:00.000,0.0,40.0,0.00,6,3,3,1,3
5323433,2025-03-09 00:00:00.000,7250270104748231,3360101417316,25000.0,25000.0,10.0,GENERAL LABOR,2025-02-09 00:00:00.000,1.0,52.0,17444.93,6,3,1,2,2
5323434,2025-03-09 00:00:00.000,7250170104643668,3800400133170,35000.0,35000.0,1.0,REGISTERED BUSINESS OWNER,2025-01-24 00:00:00.000,1.0,42.0,10775.16,6,2,2,3,3


In [293]:
data_digit1['Loan No'] = data_digit1['Loan No'].astype(str)
data_performance['CONTRACT_NO']= data_performance['CONTRACT_NO'].astype(str)

In [323]:
data_all_digit = data_performance.merge(data_digit1, left_on='CONTRACT_NO', right_on='Loan No', how='left')


In [ ]:
# data_all_digit.loc[data_all_digit['Loan No'] == '7220730100003250']

KeyError: 'Loan No'

In [ ]:
# data_all_digit['AVG_DIGIT1'] = data_all_digit.loc[:, data_all_digit.columns.str.startswith('SCORE_INDEX')].apply(pd.to_numeric, errors='coerce').mean(axis=1).fillna(0).astype(int)


In [324]:
data_all_digit

,AS_OF_DATE,CONTRACT_NO,NATIONAL_ID,GROSS_INCOME,FLOORED_VALUE,OCCUPATION_TYPE,OCCUPATION_NAME,CONTRACT_DATE,CONTRACT_PERIOD,AGE,...,DIGIT2JOB,TOTAL_DIGIT2,Loan No,BUCKET_SCORE0mnt,BUCKET_SCORE1mnt,BUCKET_SCORE2mnt,BUCKET_SCORE3mnt,BUCKET_SCORE4mnt,BUCKET_SCORE5mnt,AVG_DIGIT1
0,2025-03-09 00:00:00.000,7230950101266462,1709700073957,30000.0,30000.0,5.0,OFFICE WORK,2023-09-21 00:00:00.000,17.0,36.0,...,1,3,7230950101266462,2,1,0,5,5,4,2.833333
1,2025-03-09 00:00:00.000,5000161041003419,3300800674709,15611.0,15000.0,9.0,PRODUCTION LINE LABOR,2018-04-02 00:00:00.000,83.0,47.0,...,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-03-09 00:00:00.000,7230410100429114,1401500153197,12224.0,10000.0,9.0,PRODUCTION LINE LABOR,2023-04-25 00:00:00.000,22.0,30.0,...,3,3,7230410100429114,13,13,13,9,8,7,10.500000
3,2025-03-09 00:00:00.000,5000461011069133,1102001592652,10000.0,10000.0,6.0,SALES,2018-01-29 00:00:00.000,85.0,33.0,...,2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-03-09 00:00:00.000,5000159121088434,1100200920712,10250.0,10000.0,7.0,DRIVING,2016-12-28 00:00:00.000,98.0,30.0,...,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5323431,2025-03-09 00:00:00.000,7250270104833429,1910100097898,45000.0,45000.0,5.0,OFFICE WORK,2025-02-23 00:00:00.000,0.0,37.0,...,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5323432,2025-03-09 00:00:00.000,7250270104828777,1600100045670,17500.0,15000.0,5.0,OFFICE WORK,2025-02-23 00:00:00.000,0.0,40.0,...,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5323433,2025-03-09 00:00:00.000,7250270104748231,3360101417316,25000.0,25000.0,10.0,GENERAL LABOR,2025-02-09 00:00:00.000,1.0,52.0,...,2,2,7250270104748231,0,NaN,NaN,NaN,NaN,NaN,0.000000
5323434,2025-03-09 00:00:00.000,7250170104643668,3800400133170,35000.0,35000.0,1.0,REGISTERED BUSINESS OWNER,2025-01-24 00:00:00.000,1.0,42.0,...,3,3,7250170104643668,0,0,NaN,NaN,NaN,NaN,0.000000


In [19]:
data_all_digit.groupby('TOTAL_DIGIT2').size()

TOTAL_DIGIT2
1       4869
2    1934568
3    3383999
dtype: int64

In [325]:
data_all_digit['MOB'] = pd.to_numeric(data_all_digit['MOB'], errors='coerce').fillna(0).astype(int)

In [326]:
data_all_digit['MOB']

0          7
1          7
2          7
3          7
4          7
          ..
5323431    6
5323432    6
5323433    6
5323434    6
5323435    6
Name: MOB, Length: 5323436, dtype: int64

In [298]:
data_all_digit.columns

Index(['AS_OF_DATE', 'CONTRACT_NO', 'NATIONAL_ID', 'GROSS_INCOME',
       'FLOORED_VALUE', 'OCCUPATION_TYPE', 'OCCUPATION_NAME', 'CONTRACT_DATE',
       'CONTRACT_PERIOD', 'AGE', 'PRINCIPAL_BAL', 'MOB', 'DIGIT2INCOME',
       'DIGIT2AGE', 'DIGIT2JOB', 'TOTAL_DIGIT2', 'Loan No', 'BUCKET_SCORE0mnt',
       'BUCKET_SCORE1mnt', 'BUCKET_SCORE2mnt', 'BUCKET_SCORE3mnt',
       'BUCKET_SCORE4mnt', 'BUCKET_SCORE5mnt', 'AVG_DIGIT1'],
      dtype='object')

TDH

In [327]:
files = glob.glob("./Input/ar_all/*.*")
print(files)
    # logger.info('Start Processing...')
    # logger.info('Reading input files...')


['./Input/ar_all\\AR_ALL_20250310_1.csv', './Input/ar_all\\AR_ALL_20250310_2.csv']


In [328]:
df_list = []  # Store individual DataFrames

for file in files:
    print(f"Reading {file}...")
    df = pd.read_csv(file, encoding='cp874', low_memory=False)
    df_list.append(df)

Reading ./Input/ar_all\AR_ALL_20250310_1.csv...
Reading ./Input/ar_all\AR_ALL_20250310_2.csv...


In [329]:
data_tdr = pd.concat(df_list, ignore_index=True)
data_tdr

,H,Customer No,Loan No,Product,Project Code,Project Code Name,Promotion Code,Promotion Name,Loan Date,Maturity Date,...,AS Output Grade Code,Job Type Code,Less Than 1 Year Balance,Less Than 3 Year Balance,Less Than 5 Year Balance,Exceeding 5 Year Balance,RL Suspention Y/N,Customer Total Credit Line,SAM Y/N,SUING Y/N
0,D,"=""11200001592""","=""1000163020000364""",C4C,V002,01:C4C_068_180,,,2020-02-11,2025-03-01,...,,2,6416.62,0.00,0.0,0.0,N,140000.00,N,N
1,D,"=""11200001210""","=""1000163020001034""",C4C,V003,Re_01:C4C,,,2020-02-28,2025-03-01,...,,7,6676.18,0.00,0.0,0.0,N,290000.00,N,N
2,D,"=""11200000612""","=""1000163030000219""",C4C,V003,Re_01:C4C,,,2020-03-18,2025-04-01,...,,2,6473.42,0.00,0.0,0.0,N,280000.00,N,N
3,D,"=""11200001303""","=""1000163030000260""",C4C,V002,01:C4C_068_180,,,2020-03-25,2025-04-01,...,,9,9256.07,0.00,0.0,0.0,N,400000.00,N,N
4,D,"=""11200001023""","=""1000163030000332""",C4C,V003,Re_01:C4C,,,2020-03-31,2025-04-01,...,,2,27479.92,0.00,0.0,0.0,N,400000.00,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1279590,D,"=""13210012277""","=""9010221122000507""",TL,PLD2021070,ปรับโครงสร้างหนี้_20.9%_COVID_Phase3,,,2021-12-01,2028-01-01,...,,,16920.21,37112.42,0.0,0.0,N,205411.79,N,N
1279591,D,"=""13210012644""","=""9010221122000566""",TL,PLD2021035,ปรับโครงสร้างหนี้_จากRL_22%_COVID_Phase3,,,2021-12-01,2026-01-01,...,,,6578.74,0.00,0.0,0.0,N,45667.25,N,N
1279592,D,"=""13210001266""","=""9010221122000817""",TL,PLD2021035,ปรับโครงสร้างหนี้_จากRL_22%_COVID_Phase3,,,2021-12-01,2028-01-01,...,,,8598.03,17818.86,0.0,0.0,N,89316.84,N,N
1279593,D,"=""13210005782""","=""9010221122000931""",TL,PLD2020023,ปรับโครงสร้างหนี้_จากRL_22%_48ด,,,2021-12-08,2025-12-01,...,,,12544.04,0.00,0.0,0.0,N,106159.76,N,N


In [330]:
import re
data_tdr['Loan No'] = data_tdr['Loan No'].astype(str).apply(lambda x: re.sub(r'\D', '', x))

In [331]:
data_tdr

,H,Customer No,Loan No,Product,Project Code,Project Code Name,Promotion Code,Promotion Name,Loan Date,Maturity Date,...,AS Output Grade Code,Job Type Code,Less Than 1 Year Balance,Less Than 3 Year Balance,Less Than 5 Year Balance,Exceeding 5 Year Balance,RL Suspention Y/N,Customer Total Credit Line,SAM Y/N,SUING Y/N
0,D,"=""11200001592""",1000163020000364,C4C,V002,01:C4C_068_180,,,2020-02-11,2025-03-01,...,,2,6416.62,0.00,0.0,0.0,N,140000.00,N,N
1,D,"=""11200001210""",1000163020001034,C4C,V003,Re_01:C4C,,,2020-02-28,2025-03-01,...,,7,6676.18,0.00,0.0,0.0,N,290000.00,N,N
2,D,"=""11200000612""",1000163030000219,C4C,V003,Re_01:C4C,,,2020-03-18,2025-04-01,...,,2,6473.42,0.00,0.0,0.0,N,280000.00,N,N
3,D,"=""11200001303""",1000163030000260,C4C,V002,01:C4C_068_180,,,2020-03-25,2025-04-01,...,,9,9256.07,0.00,0.0,0.0,N,400000.00,N,N
4,D,"=""11200001023""",1000163030000332,C4C,V003,Re_01:C4C,,,2020-03-31,2025-04-01,...,,2,27479.92,0.00,0.0,0.0,N,400000.00,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1279590,D,"=""13210012277""",9010221122000507,TL,PLD2021070,ปรับโครงสร้างหนี้_20.9%_COVID_Phase3,,,2021-12-01,2028-01-01,...,,,16920.21,37112.42,0.0,0.0,N,205411.79,N,N
1279591,D,"=""13210012644""",9010221122000566,TL,PLD2021035,ปรับโครงสร้างหนี้_จากRL_22%_COVID_Phase3,,,2021-12-01,2026-01-01,...,,,6578.74,0.00,0.0,0.0,N,45667.25,N,N
1279592,D,"=""13210001266""",9010221122000817,TL,PLD2021035,ปรับโครงสร้างหนี้_จากRL_22%_COVID_Phase3,,,2021-12-01,2028-01-01,...,,,8598.03,17818.86,0.0,0.0,N,89316.84,N,N
1279593,D,"=""13210005782""",9010221122000931,TL,PLD2020023,ปรับโครงสร้างหนี้_จากRL_22%_48ด,,,2021-12-08,2025-12-01,...,,,12544.04,0.00,0.0,0.0,N,106159.76,N,N


In [332]:
data_tdr = data_tdr.loc[
    (data_tdr['Product'] == 'TL') & 
    (
        data_tdr['Project Code Name'].str.contains('ปรับโครงสร้าง', na=False) | 
        data_tdr['Project Code Name'].str.contains('TDR', na=False) | 
        data_tdr['Project Code Name'].str.contains('DR', na=False)
    )
].reset_index(drop=True)[['Loan No','Project Code Name']]

In [333]:
data_tdr

,Loan No,Project Code Name
0,7220150100000141,TDR
1,7220150100000146,TDR
2,7220150100000148,TDR
3,7220150100000152,TDR
4,7220150100000161,TDR
...,...,...
3379,9010221122000507,ปรับโครงสร้างหนี้_20.9%_COVID_Phase3
3380,9010221122000566,ปรับโครงสร้างหนี้_จากRL_22%_COVID_Phase3
3381,9010221122000817,ปรับโครงสร้างหนี้_จากRL_22%_COVID_Phase3
3382,9010221122000931,ปรับโครงสร้างหนี้_จากRL_22%_48ด


merge

In [334]:
data_cscore = data_all_digit.merge(data_tdr, left_on='CONTRACT_NO', right_on='Loan No', how='left')

In [335]:
data_cscore.columns

Index(['AS_OF_DATE', 'CONTRACT_NO', 'NATIONAL_ID', 'GROSS_INCOME',
       'FLOORED_VALUE', 'OCCUPATION_TYPE', 'OCCUPATION_NAME', 'CONTRACT_DATE',
       'CONTRACT_PERIOD', 'AGE', 'PRINCIPAL_BAL', 'MOB', 'DIGIT2INCOME',
       'DIGIT2AGE', 'DIGIT2JOB', 'TOTAL_DIGIT2', 'Loan No_x',
       'BUCKET_SCORE0mnt', 'BUCKET_SCORE1mnt', 'BUCKET_SCORE2mnt',
       'BUCKET_SCORE3mnt', 'BUCKET_SCORE4mnt', 'BUCKET_SCORE5mnt',
       'AVG_DIGIT1', 'Loan No_y', 'Project Code Name'],
      dtype='object')

In [336]:
# Define conditions
conditions = [
    data_cscore['Project Code Name'].notna(),      # A is not null
    data_cscore['MOB'] < 7,          # A is less than 7
    data_cscore['AVG_DIGIT1'] > 3  ,
    data_cscore['AVG_DIGIT1'] > 1,# C is equal to 3
    data_cscore['AVG_DIGIT1'] >= 0,
]

# Define corresponding values
choices = ['H', 'D', 'H','M','L']

data_cscore['TOTAL_DIGIT1']= np.select(conditions, choices, default='Other')

In [337]:
data_cscore = data_cscore[['AS_OF_DATE','CONTRACT_NO','GROSS_INCOME','OCCUPATION_NAME','CONTRACT_DATE','CONTRACT_PERIOD',
                'AGE','MOB','DIGIT2INCOME',
             'DIGIT2AGE', 'DIGIT2JOB', 'TOTAL_DIGIT2', 'BUCKET_SCORE0mnt', 'BUCKET_SCORE1mnt', 'BUCKET_SCORE2mnt',
       'BUCKET_SCORE3mnt', 'BUCKET_SCORE4mnt', 'BUCKET_SCORE5mnt', 'AVG_DIGIT1','TOTAL_DIGIT1', 'Project Code Name' ]]

In [338]:
data_cscore.columns

Index(['AS_OF_DATE', 'CONTRACT_NO', 'GROSS_INCOME', 'OCCUPATION_NAME',
       'CONTRACT_DATE', 'CONTRACT_PERIOD', 'AGE', 'MOB', 'DIGIT2INCOME',
       'DIGIT2AGE', 'DIGIT2JOB', 'TOTAL_DIGIT2', 'BUCKET_SCORE0mnt',
       'BUCKET_SCORE1mnt', 'BUCKET_SCORE2mnt', 'BUCKET_SCORE3mnt',
       'BUCKET_SCORE4mnt', 'BUCKET_SCORE5mnt', 'AVG_DIGIT1', 'TOTAL_DIGIT1',
       'Project Code Name'],
      dtype='object')

In [339]:
data_cscore

,AS_OF_DATE,CONTRACT_NO,GROSS_INCOME,OCCUPATION_NAME,CONTRACT_DATE,CONTRACT_PERIOD,AGE,MOB,DIGIT2INCOME,DIGIT2AGE,...,TOTAL_DIGIT2,BUCKET_SCORE0mnt,BUCKET_SCORE1mnt,BUCKET_SCORE2mnt,BUCKET_SCORE3mnt,BUCKET_SCORE4mnt,BUCKET_SCORE5mnt,AVG_DIGIT1,TOTAL_DIGIT1,Project Code Name
0,2025-03-09 00:00:00.000,7230950101266462,30000.0,OFFICE WORK,2023-09-21 00:00:00.000,17.0,36.0,7,3,3,...,3,2,1,0,5,5,4,2.833333,H,ปรับโครงสร้างหนี้_DR_Collection_15%
1,2025-03-09 00:00:00.000,5000161041003419,15611.0,PRODUCTION LINE LABOR,2018-04-02 00:00:00.000,83.0,47.0,7,3,2,...,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other,NaN
2,2025-03-09 00:00:00.000,7230410100429114,12224.0,PRODUCTION LINE LABOR,2023-04-25 00:00:00.000,22.0,30.0,7,3,3,...,3,13,13,13,9,8,7,10.500000,H,NaN
3,2025-03-09 00:00:00.000,5000461011069133,10000.0,SALES,2018-01-29 00:00:00.000,85.0,33.0,7,3,3,...,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other,NaN
4,2025-03-09 00:00:00.000,5000159121088434,10250.0,DRIVING,2016-12-28 00:00:00.000,98.0,30.0,7,3,3,...,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5323431,2025-03-09 00:00:00.000,7250270104833429,45000.0,OFFICE WORK,2025-02-23 00:00:00.000,0.0,37.0,6,2,3,...,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,NaN
5323432,2025-03-09 00:00:00.000,7250270104828777,17500.0,OFFICE WORK,2025-02-23 00:00:00.000,0.0,40.0,6,3,3,...,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,NaN
5323433,2025-03-09 00:00:00.000,7250270104748231,25000.0,GENERAL LABOR,2025-02-09 00:00:00.000,1.0,52.0,6,3,1,...,2,0,NaN,NaN,NaN,NaN,NaN,0.000000,D,NaN
5323434,2025-03-09 00:00:00.000,7250170104643668,35000.0,REGISTERED BUSINESS OWNER,2025-01-24 00:00:00.000,1.0,42.0,6,2,2,...,3,0,0,NaN,NaN,NaN,NaN,0.000000,D,NaN


In [340]:
# data_all_digit['score_digit1'] = data_all_digit['TOTAL_DIGIT1'].map({3: 'H', 2: 'M', 1: 'L'})
data_cscore['TOTAL_DIGIT2'] = data_cscore['TOTAL_DIGIT2'].map({3: 'H', 2: 'M', 1: 'L'})

In [341]:
data_cscore

,AS_OF_DATE,CONTRACT_NO,GROSS_INCOME,OCCUPATION_NAME,CONTRACT_DATE,CONTRACT_PERIOD,AGE,MOB,DIGIT2INCOME,DIGIT2AGE,...,TOTAL_DIGIT2,BUCKET_SCORE0mnt,BUCKET_SCORE1mnt,BUCKET_SCORE2mnt,BUCKET_SCORE3mnt,BUCKET_SCORE4mnt,BUCKET_SCORE5mnt,AVG_DIGIT1,TOTAL_DIGIT1,Project Code Name
0,2025-03-09 00:00:00.000,7230950101266462,30000.0,OFFICE WORK,2023-09-21 00:00:00.000,17.0,36.0,7,3,3,...,H,2,1,0,5,5,4,2.833333,H,ปรับโครงสร้างหนี้_DR_Collection_15%
1,2025-03-09 00:00:00.000,5000161041003419,15611.0,PRODUCTION LINE LABOR,2018-04-02 00:00:00.000,83.0,47.0,7,3,2,...,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other,NaN
2,2025-03-09 00:00:00.000,7230410100429114,12224.0,PRODUCTION LINE LABOR,2023-04-25 00:00:00.000,22.0,30.0,7,3,3,...,H,13,13,13,9,8,7,10.500000,H,NaN
3,2025-03-09 00:00:00.000,5000461011069133,10000.0,SALES,2018-01-29 00:00:00.000,85.0,33.0,7,3,3,...,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other,NaN
4,2025-03-09 00:00:00.000,5000159121088434,10250.0,DRIVING,2016-12-28 00:00:00.000,98.0,30.0,7,3,3,...,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5323431,2025-03-09 00:00:00.000,7250270104833429,45000.0,OFFICE WORK,2025-02-23 00:00:00.000,0.0,37.0,6,2,3,...,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,NaN
5323432,2025-03-09 00:00:00.000,7250270104828777,17500.0,OFFICE WORK,2025-02-23 00:00:00.000,0.0,40.0,6,3,3,...,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,NaN
5323433,2025-03-09 00:00:00.000,7250270104748231,25000.0,GENERAL LABOR,2025-02-09 00:00:00.000,1.0,52.0,6,3,1,...,M,0,NaN,NaN,NaN,NaN,NaN,0.000000,D,NaN
5323434,2025-03-09 00:00:00.000,7250170104643668,35000.0,REGISTERED BUSINESS OWNER,2025-01-24 00:00:00.000,1.0,42.0,6,2,2,...,H,0,0,NaN,NaN,NaN,NaN,0.000000,D,NaN


In [342]:
data_cscore['FINAL_SCORE'] =data_cscore['TOTAL_DIGIT1'].astype(str) + data_cscore['TOTAL_DIGIT2'].astype(str) +'0'+ data_cscore['MOB'].astype(str)

In [343]:
data_cscore

,AS_OF_DATE,CONTRACT_NO,GROSS_INCOME,OCCUPATION_NAME,CONTRACT_DATE,CONTRACT_PERIOD,AGE,MOB,DIGIT2INCOME,DIGIT2AGE,...,BUCKET_SCORE0mnt,BUCKET_SCORE1mnt,BUCKET_SCORE2mnt,BUCKET_SCORE3mnt,BUCKET_SCORE4mnt,BUCKET_SCORE5mnt,AVG_DIGIT1,TOTAL_DIGIT1,Project Code Name,FINAL_SCORE
0,2025-03-09 00:00:00.000,7230950101266462,30000.0,OFFICE WORK,2023-09-21 00:00:00.000,17.0,36.0,7,3,3,...,2,1,0,5,5,4,2.833333,H,ปรับโครงสร้างหนี้_DR_Collection_15%,HH07
1,2025-03-09 00:00:00.000,5000161041003419,15611.0,PRODUCTION LINE LABOR,2018-04-02 00:00:00.000,83.0,47.0,7,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other,NaN,OtherH07
2,2025-03-09 00:00:00.000,7230410100429114,12224.0,PRODUCTION LINE LABOR,2023-04-25 00:00:00.000,22.0,30.0,7,3,3,...,13,13,13,9,8,7,10.500000,H,NaN,HH07
3,2025-03-09 00:00:00.000,5000461011069133,10000.0,SALES,2018-01-29 00:00:00.000,85.0,33.0,7,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other,NaN,OtherH07
4,2025-03-09 00:00:00.000,5000159121088434,10250.0,DRIVING,2016-12-28 00:00:00.000,98.0,30.0,7,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other,NaN,OtherH07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5323431,2025-03-09 00:00:00.000,7250270104833429,45000.0,OFFICE WORK,2025-02-23 00:00:00.000,0.0,37.0,6,2,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,NaN,DM06
5323432,2025-03-09 00:00:00.000,7250270104828777,17500.0,OFFICE WORK,2025-02-23 00:00:00.000,0.0,40.0,6,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,NaN,DH06
5323433,2025-03-09 00:00:00.000,7250270104748231,25000.0,GENERAL LABOR,2025-02-09 00:00:00.000,1.0,52.0,6,3,1,...,0,NaN,NaN,NaN,NaN,NaN,0.000000,D,NaN,DM06
5323434,2025-03-09 00:00:00.000,7250170104643668,35000.0,REGISTERED BUSINESS OWNER,2025-01-24 00:00:00.000,1.0,42.0,6,2,2,...,0,0,NaN,NaN,NaN,NaN,0.000000,D,NaN,DH06


In [344]:
data_cscore.groupby('FINAL_SCORE').size()

FINAL_SCORE
DH06         829980
DL06            631
DM06         579392
HH06            767
HH07          90686
HL06             29
HL07            439
HM06            468
HM07          31085
LH07         571811
LL07           2085
LM07         402998
MH07          10556
ML07             50
MM07           5924
OtherH07    1880199
OtherL07       1635
OtherM07     914701
dtype: int64

=====ย้อนหลัง====

In [345]:
data_cscore.columns

Index(['AS_OF_DATE', 'CONTRACT_NO', 'GROSS_INCOME', 'OCCUPATION_NAME',
       'CONTRACT_DATE', 'CONTRACT_PERIOD', 'AGE', 'MOB', 'DIGIT2INCOME',
       'DIGIT2AGE', 'DIGIT2JOB', 'TOTAL_DIGIT2', 'BUCKET_SCORE0mnt',
       'BUCKET_SCORE1mnt', 'BUCKET_SCORE2mnt', 'BUCKET_SCORE3mnt',
       'BUCKET_SCORE4mnt', 'BUCKET_SCORE5mnt', 'AVG_DIGIT1', 'TOTAL_DIGIT1',
       'Project Code Name', 'FINAL_SCORE'],
      dtype='object')

==

mapassign

In [346]:
data_assign = pd.read_excel("./input/assisgn_data/3.Total assign Mar'25 (1).xlsx")

In [347]:
file = glob.glob("./input/assisgn_data/*.xlsx") 
data_assign = pd.read_excel(file[0])

In [348]:
file[0].split('\\')[1]

"3.Total assign Mar'25 (1).xlsx"

In [349]:
data_assign = data_assign[['Loan No.','Old OA','New OA']]
data_assign

,Loan No.,Old OA,New OA
0,7220730100003250,KOR,KOR
1,7221030100003286,NaN,Delay calling
2,7230340100241933,Delay calling,Delay calling
3,7230530100570947,PCN,KOR
4,7230940101310936,NaN,Delay calling
...,...,...,...
40416,7240450102627362,KWC,KWC
40417,7240550102879930,GMV,MMN
40418,7240750103341461,KWC,KWC
40419,7240650103061703,MMN,KWC


In [352]:
data_assign['Loan No.'] = data_assign['Loan No.'].astype(str)

In [354]:
data_assign

,Loan No.,Old OA,New OA
0,7220730100003250,KOR,KOR
1,7221030100003286,NaN,Delay calling
2,7230340100241933,Delay calling,Delay calling
3,7230530100570947,PCN,KOR
4,7230940101310936,NaN,Delay calling
...,...,...,...
40416,7240450102627362,KWC,KWC
40417,7240550102879930,GMV,MMN
40418,7240750103341461,KWC,KWC
40419,7240650103061703,MMN,KWC


In [353]:
data_cscore

,AS_OF_DATE,CONTRACT_NO,GROSS_INCOME,OCCUPATION_NAME,CONTRACT_DATE,CONTRACT_PERIOD,AGE,MOB,DIGIT2INCOME,DIGIT2AGE,...,BUCKET_SCORE0mnt,BUCKET_SCORE1mnt,BUCKET_SCORE2mnt,BUCKET_SCORE3mnt,BUCKET_SCORE4mnt,BUCKET_SCORE5mnt,AVG_DIGIT1,TOTAL_DIGIT1,Project Code Name,FINAL_SCORE
0,2025-03-09 00:00:00.000,7230950101266462,30000.0,OFFICE WORK,2023-09-21 00:00:00.000,17.0,36.0,7,3,3,...,2,1,0,5,5,4,2.833333,H,ปรับโครงสร้างหนี้_DR_Collection_15%,HH07
1,2025-03-09 00:00:00.000,5000161041003419,15611.0,PRODUCTION LINE LABOR,2018-04-02 00:00:00.000,83.0,47.0,7,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other,NaN,OtherH07
2,2025-03-09 00:00:00.000,7230410100429114,12224.0,PRODUCTION LINE LABOR,2023-04-25 00:00:00.000,22.0,30.0,7,3,3,...,13,13,13,9,8,7,10.500000,H,NaN,HH07
3,2025-03-09 00:00:00.000,5000461011069133,10000.0,SALES,2018-01-29 00:00:00.000,85.0,33.0,7,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other,NaN,OtherH07
4,2025-03-09 00:00:00.000,5000159121088434,10250.0,DRIVING,2016-12-28 00:00:00.000,98.0,30.0,7,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other,NaN,OtherH07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5323431,2025-03-09 00:00:00.000,7250270104833429,45000.0,OFFICE WORK,2025-02-23 00:00:00.000,0.0,37.0,6,2,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,NaN,DM06
5323432,2025-03-09 00:00:00.000,7250270104828777,17500.0,OFFICE WORK,2025-02-23 00:00:00.000,0.0,40.0,6,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,NaN,DH06
5323433,2025-03-09 00:00:00.000,7250270104748231,25000.0,GENERAL LABOR,2025-02-09 00:00:00.000,1.0,52.0,6,3,1,...,0,NaN,NaN,NaN,NaN,NaN,0.000000,D,NaN,DM06
5323434,2025-03-09 00:00:00.000,7250170104643668,35000.0,REGISTERED BUSINESS OWNER,2025-01-24 00:00:00.000,1.0,42.0,6,2,2,...,0,0,NaN,NaN,NaN,NaN,0.000000,D,NaN,DH06


In [355]:
cscore_assigned = data_assign.merge(data_cscore,left_on='Loan No.',right_on='CONTRACT_NO',how='left')

In [356]:
cscore_assigned.columns

Index(['Loan No.', 'Old OA', 'New OA', 'AS_OF_DATE', 'CONTRACT_NO',
       'GROSS_INCOME', 'OCCUPATION_NAME', 'CONTRACT_DATE', 'CONTRACT_PERIOD',
       'AGE', 'MOB', 'DIGIT2INCOME', 'DIGIT2AGE', 'DIGIT2JOB', 'TOTAL_DIGIT2',
       'BUCKET_SCORE0mnt', 'BUCKET_SCORE1mnt', 'BUCKET_SCORE2mnt',
       'BUCKET_SCORE3mnt', 'BUCKET_SCORE4mnt', 'BUCKET_SCORE5mnt',
       'AVG_DIGIT1', 'TOTAL_DIGIT1', 'Project Code Name', 'FINAL_SCORE'],
      dtype='object')

In [357]:
cscore_assigned

,Loan No.,Old OA,New OA,AS_OF_DATE,CONTRACT_NO,GROSS_INCOME,OCCUPATION_NAME,CONTRACT_DATE,CONTRACT_PERIOD,AGE,...,BUCKET_SCORE0mnt,BUCKET_SCORE1mnt,BUCKET_SCORE2mnt,BUCKET_SCORE3mnt,BUCKET_SCORE4mnt,BUCKET_SCORE5mnt,AVG_DIGIT1,TOTAL_DIGIT1,Project Code Name,FINAL_SCORE
0,7220730100003250,KOR,KOR,2025-03-09 00:00:00.000,7220730100003250,12900.0,DRIVING,2022-07-12 00:00:00.000,31.0,44.0,...,0,0,0,0,0,0,0.000000,L,NaN,LH07
1,7221030100003286,NaN,Delay calling,2025-03-09 00:00:00.000,7221030100003286,42000.0,OFFICE WORK,2022-10-12 00:00:00.000,28.0,45.0,...,0,0,0,0,0,0,0.000000,L,NaN,LM07
2,7230340100241933,Delay calling,Delay calling,2025-03-09 00:00:00.000,7230340100241933,17125.0,DRIVING,2023-03-06 00:00:00.000,24.0,58.0,...,0,0,0,0,0,0,0.000000,L,NaN,LH07
3,7230530100570947,PCN,KOR,2025-03-09 00:00:00.000,7230530100570947,23290.0,OFFICE WORK,2023-05-23 00:00:00.000,21.0,50.0,...,2,1,0,0,0,0,0.500000,L,NaN,LM07
4,7230940101310936,NaN,Delay calling,2025-03-09 00:00:00.000,7230940101310936,45000.0,OFFICE WORK,2023-09-29 00:00:00.000,17.0,42.0,...,0,0,0,0,0,0,0.000000,L,NaN,LM07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40416,7240450102627362,KWC,KWC,2025-03-09 00:00:00.000,7240450102627362,11195.0,GENERAL LABOR,2024-04-23 00:00:00.000,10.0,48.0,...,6,5,5,5,4,3,4.666667,H,ปรับโครงสร้างหนี้_DR_Collection_15%,HH07
40417,7240550102879930,GMV,MMN,2025-03-09 00:00:00.000,7240550102879930,11000.0,OFFICE WORK,2024-05-28 00:00:00.000,9.0,40.0,...,6,5,4,3,2,1,3.500000,H,ปรับโครงสร้างหนี้_TDR_Collection_15%,HH07
40418,7240750103341461,KWC,KWC,2025-03-09 00:00:00.000,7240750103341461,9728.0,OFFICE WORK,2024-07-24 00:00:00.000,7.0,39.0,...,6,5,4,3,2,1,3.500000,H,ปรับโครงสร้างหนี้_PD_Collection_15%,HH07
40419,7240650103061703,MMN,KWC,2025-03-09 00:00:00.000,7240650103061703,9569.0,GENERAL LABOR,2024-06-18 00:00:00.000,8.0,50.0,...,6,5,4,3,2,1,3.500000,H,ปรับโครงสร้างหนี้_DR_Collection_15%,HH07


In [361]:
file = glob.glob("./input/assisgn_data/*.xlsx") 
file_path = file[0].split('\\')[1]


In [ ]:
cscore_assigned.to_csv(f'./output/assign_{file_path}.csv')

: 